In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time


class Scraper:
    def __init__(self, base_url, path, company, filename):
        self.filename = filename
        self.base_url = base_url
        self.company = company
        self.driver = webdriver.Chrome(path)
        self.links = []

    def run(self):
        self.driver.get(self.base_url)
        self.search_company()
        self.get_all_reviews()
        self.reach_links()

    def reach_links(self):
        for link in self.links:
            self.driver.get(link)
            time.sleep(1)
            self.get_data()

    def get_data(self):
        lst = [self.driver.find_element_by_class_name("_1xPYIUfA9IwtVyBsOt_SaR").text,
               self.driver.find_element_by_class_name("_1_dikBfyioZMrCoMA4C9hZ").text]
        try:
            lst.append(self.driver.find_element_by_id("years-worked-with").text)
        except:
            lst.append("None")
        try:
            lst.append(self.driver.find_element_by_id("work-location").text)
        except:
            lst.append("None")
        stars = self.driver.find_elements_by_class_name("OX0_TZZSlhMhIywgR8jko")
        lst.append(stars[0].text)
        lst.append(stars[2].text)
        lst.append(stars[4].text)
        lst.append(stars[1].text)
        lst.append(stars[3].text)
        lst.append(stars[5].text)
        lst.append(self.driver.find_element_by_class_name("Tlh9F04dzRFo78gJPlhMM").text)
        lst.append(self.driver.find_element_by_class_name("_3MyALCcnLdxQbK6kunFzgk").text)
        lst.append(self.driver.find_element_by_id("good-review").text)
        lst.append(self.driver.find_element_by_id("challange-review").text)
        divs = self.driver.find_elements_by_class_name("_2N4PqnlVAzHqUH2g5yZLn5")
        lst.append(divs[0].text)
        lst.append(divs[1].text)

        self.append_data(lst)

    def append_data(self, data):
        with open(self.filename, 'a') as f:
            f.write("\n")
            for d in data:
                try:
                    f.write(d + ",")
                except:
                    f.write("None" + ",")

    def get_all_reviews(self):
        pages = self.get_pages_number()
        for page in range(pages):
            self.get_review_links()
            if page + 1 != pages:
                self.go_to_next_page()

    def go_to_next_page(self):
        self.driver.find_element_by_class_name("_1obKPTnJFYAW7RWRamBIS5").click()

    def get_pages_number(self):
        time.sleep(2)
        div = self.driver.find_element_by_class_name("RZ46YESzIdDEUyK1AEEUQ")
        spans = div.find_elements_by_tag_name("span")
        pages = int(spans[2].text.split(" ")[-1])
        return pages

    def get_review_links(self):
        time.sleep(2)
        parents = self.driver.find_elements_by_class_name("_3MyALCcnLdxQbK6kunFzgk")
        for parent in parents:
            a = parent.find_element_by_tag_name("a")
            self.links.append(a.get_attribute("href"))
        return

    def search_company(self):
        search_input = self.driver.find_element_by_id("companySearch")
        search_input.send_keys(self.company)
        search_input.send_keys(Keys.ENTER)
        time.sleep(2)
        self.driver.find_element_by_class_name("tNpZ-r8HSFPRZ6NJvAkbQ").click()
        time.sleep(2)
        self.driver.find_element_by_class_name("_3Z7Y99Bn3qT4N_dKKyG0ka").click()
        return


def make_csv_file(filename):
    with open(filename, "w") as f:
        f.write(
            "Job title,review data,duration,home location,benefits & perks,work/life balance,management,career development,working environment,diversity & equal opportunity,overall,review title,the good things,the challenges,salary rated,employer_recommendation")


if __name__ == '__main__':
    PATH = "chromedriver.exe"
    BASE_URL = "https://www.seek.com.au/companies"
    COMPANY = "H&R BLOCK"  # Here comes company name to search for
    FILENAME = "data.csv"  # The filename of data
    make_csv_file(FILENAME)
    Scraper(BASE_URL, PATH, COMPANY, FILENAME).run()
